# CloudMasking

In [ ]:
import arcpy
import pandas as pd
import os
arcpy.env.parallelProcessingFactor = "100%"
arcpy.env.overwriteOutput = True

In [ ]:
table = r'C:\Users\rob10341\Documents\NuveenNC\NNC_Sen2_Transfer.gdb\Sen2_NNC_2023_AOI02_WGS84_4326_RedRiverPine'
out_dir = r'C:\Users\rob10341\Documents\NuveenNC\cloud_proc'
dlpk = r'C:\Users\rob10341\Downloads\cloud_mask.dlpk'

In [ ]:
fields = ['GroupName']
df = pd.DataFrame(arcpy.da.TableToNumPyArray(table, fields))
scene_list = df['GroupName'].to_list()

for scene in scene_list:
    print(scene)
    scene_name = scene + '_'
    sel_stmt = "GroupName = '{}'".format(scene)
    print(sel_stmt)

    out_scene_name = scene.replace('.SAFE','')
    
    ref_MDS = table + '_' + out_scene_name
    
    
    scl_raster_name = os.path.join(out_dir, out_scene_name + '_SCL.TIF')
    scl_raster_reclass_name = os.path.join(out_dir, out_scene_name + '_SCL_class.TIF')
    dlpk_class_raster_name = os.path.join(out_dir, out_scene_name + '_dlpk_class.TIF')
    comb_mask_name = os.path.join(out_dir, out_scene_name + '_comb_mask.TIF') 
    comb_mask_int_name = os.path.join(out_dir, out_scene_name + '_comb_mask_int.TIF')
    comb_mask_int_expand_name = os.path.join(out_dir, out_scene_name + '_comb_mask_int_ex.TIF')
    comb_mask_int_expand_name_reclass_name = os.path.join(out_dir, out_scene_name + '_comb_mask_int_ex_class.TIF')
    comb_mask_int_expand_stacked_name = os.path.join(out_dir, out_scene_name + '_comb_mask_int_ex_class_stacked.TIF')
    cloud_masked_name = os.path.join(out_dir, out_scene_name + '_cloud_masked.TIF') 
    
    arcpy.management.CreateReferencedMosaicDataset(
        in_dataset=table,
        out_mosaic_dataset=ref_MDS,
        coordinate_system='PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]]',
        number_of_bands=None,
        pixel_type="",
        where_clause=sel_stmt,
        in_template_dataset=None,
        extent=None,
        select_using_features="SELECT_USING_FEATURES",
        lod_field=None,
        minPS_field=None,
        maxPS_field=None,
        pixelSize=None,
        build_boundary="BUILD_BOUNDARY"
    )
    
    
    # extract the SCL
    scl_raster = arcpy.management.MakeRasterLayer(ref_MDS, 
                                             out_scene_name+"_temp", 
                                             band_index = "15")
    arcpy.management.CopyRaster(scl_raster, 
                                scl_raster_name, 
                                '', 
                                None, 
                                "255", 
                                "NONE", 
                                "NONE", 
                                '', 
                                "NONE", 
                                "NONE", 
                                "TIFF", 
                                "NONE", 
                                "CURRENT_SLICE", 
                                "NO_TRANSPOSE")
    print('Extracted scl raster {0}'.format(scl_raster_name))
    
    # Reclassify the SCL
    ras_SCL_reclass = arcpy.sa.Reclassify(scl_raster, 
                                     "Value", 
                                     "0 0;1 0;2 1;3 1;4 0;5 0;6 0;7 0;8 1;9 1;10 1;11 1;NODATA 0", 
                                     "DATA")
    ras_SCL_reclass.save(scl_raster_reclass_name)
    print('Reclassified scl raster {0}'.format(scl_raster_reclass_name))

    # Create cloud mask from DLPK
    with arcpy.EnvManager(processorType="GPU"):
        cloud_ras_dlpk = arcpy.ia.ClassifyPixelsUsingDeepLearning(ref_MDS,
                                                                  dlpk, 
                                                                  "padding 64;batch_size 20;predict_background True;test_time_augmentation False;tile_size 256", 
                                                                  "PROCESS_AS_MOSAICKED_IMAGE", 
                                                                  None)
        print('Classified {0} raster pixels using cloud dlpk'.format(ref_MDS))
    # Reclassify cloud mask from DLPK
    cloud_ras_dlpk_reclass = arcpy.sa.Reclassify(cloud_ras_dlpk, 
                                     "Value", 
                                     "1 1;2 1;3 1;NODATA 0", 
                                     "DATA")
    cloud_ras_dlpk_reclass.save(dlpk_class_raster_name)
    print('Reclassified dlpk cloudmask {0} raster pixels using cloud dlpk'.format(dlpk_class_raster_name))

    # Combine the cloud masks
    input_rasters = []
    input_rasters.append(scl_raster_reclass_name)
    input_rasters.append(dlpk_class_raster_name)    
    con_stmt = 'Con(Ras1 == 1, 1, Con(Ras2 == 1, 1, 0))'
    comb_mask = arcpy.ia.RasterCalculator(input_rasters, ['Ras1', 'Ras2'] , con_stmt)
    comb_mask.save(comb_mask_name)
    print('Combined cloud masks: scl {0} and dlpk {1}'.format(scl_raster_reclass_name, dlpk_class_raster_name))

    # Convert the combined cloud masks to int
    comb_mask_int = arcpy.ia.Int(comb_mask)
    comb_mask_int.save(comb_mask_int_name)
    print('Converted combined cloud mask {0} to int'.format(comb_mask_int_name))

    # Expand the cloud mask int
    comb_mask_int_expand = arcpy.sa.Expand(comb_mask_int_name, 40, [1], "MORPHOLOGICAL")
    comb_mask_int_expand.save(comb_mask_int_expand_name)
    print('Expanded combined cloud mask {0}'.format(comb_mask_int_expand_name))


    # Reclassify the expanded cloud mask int
    comb_mask_int_expand_reclass = arcpy.sa.Reclassify(comb_mask_int_expand_name, 
                                     "Value", 
                                     "0 1;1 0;NODATA 0", 
                                     "DATA")
    comb_mask_int_expand_reclass.save(comb_mask_int_expand_name_reclass_name)
    print('Reclassified expanded cloudmask {0}'.format(comb_mask_int_expand_name_reclass_name))


    # Composite the bands of the final cloud mask
    composite_ras_list = []
    composite_ras_list.append(comb_mask_int_expand_name_reclass_name)
    composite_ras_list = composite_ras_list * 15
    cloud_ras_stmt = ';'.join(composite_ras_list)


    with arcpy.EnvManager(parallelProcessingFactor="0%"):
        arcpy.management.CompositeBands(cloud_ras_stmt,
                                        comb_mask_int_expand_stacked_name)
        print('Created final cloud mask {0}'.format(comb_mask_int_expand_stacked_name))

    # Apply mask
    cloud_masked = arcpy.ia.Times(ref_MDS,
                                  comb_mask_int_expand_stacked_name) 
    cloud_masked.save(cloud_masked_name)
    print('Masked input scene {0}'.format(cloud_masked_name))

    arcpy.management.SetRasterProperties(cloud_masked_name, 
                                 '',
                                 None, 
                                 None, 
                                 "1 0;2 0;3 0;4 0;5 0;6 0;7 0;8 0;9 0;10 0;11 0;12 0;13 0;14 0", 
                                 None, 
                                 None)
    print('Set NoData to 0 for {0}'.format(cloud_masked_name))

    # Delete temp files on disk
    #arcpy.management.Delete(raster)
    arcpy.management.Delete(scl_raster_name)
    arcpy.management.Delete(scl_raster_reclass_name)
    arcpy.management.Delete(dlpk_class_raster_name)
    arcpy.management.Delete(comb_mask_name)
    arcpy.management.Delete(comb_mask_int_name)
    arcpy.management.Delete(comb_mask_int_expand_name)
    arcpy.management.Delete(comb_mask_int_expand_name_reclass_name)
    arcpy.management.Delete(comb_mask_int_expand_stacked_name)
    arcpy.management.Delete(ref_MDS)
    
cloud_masked_md = table + '_cloud_masked'
comb_cloud_masked_tif = os.path.join(out_dir, os.path.basename(cloud_masked_md) + '_comb.tif')
comb_median_tif = os.path.join(out_dir, os.path.basename(table) + '_mdn_comb.tif')

# Create Mosaic Dataset for the cloud masked scenes
arcpy.management.CreateMosaicDataset(
    in_workspace=os.path.dirname(cloud_masked_md),
    in_mosaicdataset_name=os.path.basename(cloud_masked_md),
    coordinate_system='PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",\
    GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",\
    SPHEROID["WGS_1984",6378137.0,298.257223563]],\
    PRIMEM["Greenwich",0.0],\
    UNIT["Degree",0.0174532925199433]],\
    PROJECTION["Mercator_Auxiliary_Sphere"],\
    PARAMETER["False_Easting",0.0],\
    PARAMETER["False_Northing",0.0],\
    PARAMETER["Central_Meridian",0.0],\
    PARAMETER["Standard_Parallel_1",0.0],\
    PARAMETER["Auxiliary_Sphere_Type",0.0],\
    UNIT["Meter",1.0]]'
)
print('Created mosaic dataset {0}'.format(cloud_masked_md))

arcpy.management.AddRastersToMosaicDataset(
    in_mosaic_dataset=cloud_masked_md,
    raster_type="Raster Dataset",
    input_path=out_dir,
    filter="*_cloud_masked.tif"
)
print('Added rasters to mosaic dataset {0}'.format(cloud_masked_md))

arcpy.management.SetMosaicDatasetProperties(
    in_mosaic_dataset=cloud_masked_md,
    allowed_mosaic_methods=None,
    default_mosaic_method="None",
    mosaic_operator="MEAN",
    max_num_per_mosaic=10000,
    processing_templates="None;C:/Program Files/ArcGIS/Pro/Resources/Raster/Functions/Custom/Sentinel2_Median_BugFix.rft.xml",
    default_processing_template="C:/Program Files/ArcGIS/Pro/Resources/Raster/Functions/Custom/Sentinel2_Median_BugFix.rft.xml",
)
print('Set properties for mosaic dataset {0}'.format(cloud_masked_md))

# arcpy.management.CopyRaster(
#     in_raster=cloud_masked_md,
#     out_rasterdataset=comb_cloud_masked_tif,
#     config_keyword="",
#     background_value=None,
#     nodata_value="3.4e+38",
#     onebit_to_eightbit="NONE",
#     colormap_to_RGB="NONE",
#     pixel_type="",
#     scale_pixel_value="NONE",
#     RGB_to_Colormap="NONE",
#     format="TIFF",
#     transform="NONE",
#     process_as_multidimensional="CURRENT_SLICE",
#     build_multidimensional_transpose="NO_TRANSPOSE"
# )
# print('Copied mosaic dataset {0} to {1}'.format(cloud_masked_md,comb_cloud_masked_tif))

arcpy.management.SetMosaicDatasetProperties(
    in_mosaic_dataset=table,
    allowed_mosaic_methods=None,
    default_mosaic_method="None",
    mosaic_operator="MEAN",
    max_num_per_mosaic=10000,
    processing_templates="None;C:/Program Files/ArcGIS/Pro/Resources/Raster/Functions/Custom/Sentinel2_Median_BugFix.rft.xml",
    default_processing_template="C:/Program Files/ArcGIS/Pro/Resources/Raster/Functions/Custom/Sentinel2_Median_BugFix.rft.xml",
)
print('Set properties for mosaic dataset {0}'.format(table))

# arcpy.management.CopyRaster(
#     in_raster=table,
#     out_rasterdataset=comb_median_tif,
#     config_keyword="",
#     background_value=None,
#     nodata_value="3.4e+38",
#     onebit_to_eightbit="NONE",
#     colormap_to_RGB="NONE",
#     pixel_type="",
#     scale_pixel_value="NONE",
#     RGB_to_Colormap="NONE",
#     format="TIFF",
#     transform="NONE",
#     process_as_multidimensional="CURRENT_SLICE",
#     build_multidimensional_transpose="NO_TRANSPOSE"
# )
# print('Copied mosaic dataset {0} to {1}'.format(table,comb_median_tif))

#desc = arcpy.Describe(comb_median_tif)
#full_extent = '{} {} {} {} {}'.format(desc.extent.XMin, desc.extent.YMin, desc.extent.XMax, desc.extent.YMax, desc.spatialReference.exportToString().split(';')[0])
